In [2]:
!unzip /content/train.csv.zip

Archive:  /content/train.csv.zip
  inflating: train.csv               


In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


data = pd.read_csv('train.csv')

data = np.array(data)
m, n = data.shape
print(m)
print(n)
np.random.shuffle(data) # shuffle before splitting into dev and training sets

data_dev = data[0:1000].T
Y_dev = data_dev[0]


X_dev = data_dev[1:n]
X_dev = X_dev / 255.

#print(len(X_dev[0]))

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

42000
785


In [21]:
import numpy as np


# def init_params():
#     W1 = np.random.rand(10, 784) - 0.5
#     b1 = np.random.rand(10, 1) - 0.5
#     W2 = np.random.rand(10, 10) - 0.5
#     b2 = np.random.rand(10, 1) - 0.5
#     return W1, b1, W2, b2

def init_params():
    W1 = np.random.rand(100, 784) - 0.5
    b1 = np.random.rand(100, 1) - 0.5
    W2 = np.random.rand(100, 100) - 0.5
    b2 = np.random.rand(100, 1) - 0.5
    W3 = np.random.rand(10,100) - 0.5
    b3 = np.random.rand(10,1) - 0.5
    return W1, b1, W2, b2, W3, b3

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, W3, b3, X):
    Z1 = W1.dot(X) + b1
    print(Z1.shape)
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    print(Z2.shape)
    A2 = ReLU(Z2)
    Z3 = W3.dot(A2) + b3
    print(Z3.shape)
    A3 = softmax(Z3)
    print(A3)
    return Z1, A1, Z2, A2,Z3, A3

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2,Z3,A3, W1, W2,W3, X, Y):
    one_hot_Y = one_hot(Y)
    dZ3 = A3 - one_hot_Y
    dW3 = 1 / m * dZ3.dot(A2.T)
    db3 = 1 / m * np.sum(dZ3)
    dZ2 = W2.T.dot(dZ3) * ReLU_deriv(Z2)
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2,dW3,db3

def update_params(W1, b1, W2, b2,W3,b3, dW1, db1, dW2, db2,dW3,db3, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2
    W3 = W3 - alpha * dW3  
    b3 = b3 - alpha * db3     
    return W1, b1, W2, b2,W3,b3

def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2,W3,b3 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2,Z3,A3 = forward_prop(W1, b1, W2, b2,W3,b3, X)
        dW1, db1, dW2, db2,dW3,db3 = backward_prop(Z1, A1, Z2, A2,Z3,A3, W1, W2,W3, X, Y)
        W1, b1, W2, b2,W3,b3 = update_params(W1, b1, W2, b2,W3,b3, dW1, db1, dW2, db2,dW3,db3, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2,W3,b3


W1, b1, W2, b2, W3,b3 = gradient_descent(X_train, Y_train, 0.10, 500)





(100, 41000)
(100, 41000)
(10, 41000)
[[1.58058236e-05 7.04255266e-04 7.07245304e-04 ... 8.15494557e-02
  5.05963638e-01 1.01618043e-04]
 [3.46326204e-07 1.20992182e-06 1.31457104e-08 ... 2.29314120e-02
  2.64783689e-05 5.41284539e-12]
 [3.52951045e-13 2.53693298e-07 4.81220221e-04 ... 8.93788996e-01
  5.10061318e-06 3.07376759e-12]
 ...
 [5.21318591e-22 2.78457736e-22 7.24146525e-17 ... 4.22505182e-07
  2.14876188e-09 3.98674256e-23]
 [7.58083932e-14 4.70338053e-07 1.57723403e-06 ... 4.95997726e-08
  9.96083357e-07 1.61756578e-12]
 [1.35815417e-19 1.47892484e-10 1.96036026e-05 ... 1.67953776e-03
  1.92597060e-12 1.63416452e-15]]


ValueError: ignored

In [ ]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
get_accuracy(dev_predictions, Y_dev)


[3 3 0 5 6 5 1 3 5 9 8 0 5 9 9 4 7 9 1 6 7 4 3 8 6 9 1 4 1 4 6 8 1 7 0 5 4
 6 0 8 1 5 6 5 2 2 0 6 3 5 7 3 0 0 4 6 9 0 8 0 4 2 5 2 8 5 6 8 2 9 7 3 1 9
 9 7 2 2 1 5 3 1 1 2 4 8 0 2 1 6 5 1 0 4 1 7 1 6 5 3 9 0 3 2 6 5 6 4 7 4 4
 8 9 2 5 4 4 9 7 4 7 7 2 9 3 4 5 3 7 5 8 3 9 5 4 1 9 3 8 2 5 1 0 1 8 1 8 9
 3 9 2 8 7 9 0 0 4 0 6 4 9 8 6 6 7 6 6 6 6 0 2 5 7 2 9 8 3 0 4 2 0 7 3 8 1
 8 3 8 4 2 8 5 1 9 9 9 8 4 7 3 1 3 5 5 8 1 1 9 7 6 6 2 5 5 9 4 3 9 5 4 4 0
 5 8 7 0 4 3 2 6 8 6 9 0 9 1 8 4 2 2 4 2 1 0 7 0 0 9 4 0 1 8 4 2 0 6 1 7 7
 0 0 5 5 9 4 9 4 3 2 1 7 2 6 3 2 9 9 8 3 5 4 4 6 7 1 0 1 3 2 9 1 6 5 2 5 9
 3 6 9 9 9 3 3 8 6 4 8 8 2 3 3 8 6 0 9 0 0 9 2 9 2 3 3 5 7 0 6 2 8 2 6 1 8
 2 2 1 4 5 0 2 5 1 2 3 5 0 1 3 4 7 9 3 1 5 3 4 5 4 5 5 5 5 5 7 7 9 4 4 9 6
 6 1 6 6 0 9 6 0 0 9 9 7 4 3 2 0 1 0 9 0 9 4 1 0 0 0 8 9 7 5 4 2 7 2 3 1 7
 7 1 7 3 8 5 3 7 6 8 0 8 9 7 5 2 4 1 7 8 4 4 0 9 6 5 5 6 9 6 7 3 2 8 3 6 9
 3 9 2 1 6 3 2 1 7 9 1 2 8 3 3 7 3 4 0 3 7 3 1 2 0 5 3 0 1 3 7 8 2 2 9 9 8
 6 1 5 6 9 6 2 1 4 1 2 0 

0.89

ModuleNotFoundError: ignored

In [23]:
from sklearn.neural_network import MLPClassifier

import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import pandas
from sklearn.metrics import accuracy_score
from sklearn import naive_bayes as nb
import numpy as np
from sklearn.model_selection import train_test_split

from joblib.numpy_pickle_utils import xrange
from numpy import *

dataframe = pandas.read_table('/content/mixed200.txt')
data = dataframe.drop("No.", axis=1)
data = data.drop("StdPageRank", axis=1)
data = data.drop("VarPageRank", axis=1)
dataset = data.values



# split into input (X) and output (Y) variables
training = 57748
testing = 24749

l = 82497
X = dataset[:,0:]
# print(X.shape)
labels = []
for i in range (0,l):
  if X[i][9] == 1:
    labels.append(1)
  else:
    labels.append(0)

features = X[0:l,0:9]




# Split our data
train, test, train_labels, test_labels = features[0:training],features[training:testing+training],labels[0:training],labels[training:testing+training]


clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(7), random_state=1)

clf.fit(train,train_labels)
yPredict = clf.predict(test)

print(accuracy_score(test_labels,yPredict))


0.985615580427492


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
